In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# importa o TensorFlow
import tensorflow as tf

# pacotes adicionais 
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import math
%matplotlib inline


In [ ]:
mnist = tf.keras.datasets.mnist

In [ ]:
#if not os.path.exists('tmp'):
    #os.makedirs('tmp')
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

#n_train = x_train.num_examples # 55,000
#n_validation = mnist.validation.num_examples # 5000
#n_test = mnist.test.num_examples # 10,000


In [ ]:
n_input = 784   # input layer (28x28 pixels)
n_hidden1 = 512 # 1st hidden layer
n_hidden2 = 256 # 2nd hidden layer
n_hidden3 = 128 # 3rd hidden layer
n_output = 10   # output layer (0-9 digits)


In [ ]:
learning_rate = 1e-4
n_iterations = 1000
batch_size = 128
dropout = 0.5


In [ ]:
tf.compat.v1.disable_eager_execution()

X = tf.compat.v1.placeholder("float", [None, n_input])
Y = tf.compat.v1.placeholder("float", [None, n_output])
keep_prob = tf.compat.v1.placeholder(tf.float32)


In [ ]:
'''
weights = {
    'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1)),
}
'''

weights = {
    'w1': tf.random.truncated_normal([n_input, n_hidden1], mean=0.0, stddev=0.1, dtype=tf.dtypes.float32, seed=None, name=None),
    'w2': tf.random.truncated_normal([n_hidden1, n_hidden2], mean=0.0, stddev=0.1, dtype=tf.dtypes.float32, seed=None, name=None),
    'w3': tf.random.truncated_normal([n_hidden2, n_hidden3], mean=0.0, stddev=0.1, dtype=tf.dtypes.float32, seed=None, name=None),
    'out': tf.random.truncated_normal([n_hidden3, n_output], mean=0.0, stddev=0.1, dtype=tf.dtypes.float32, seed=None, name=None),
}

# tf.random.truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.dtypes.float32, seed=None, name=None)


In [ ]:
biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}


In [ ]:
layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
layer_drop = tf.nn.dropout(layer_3, keep_prob)
output_layer = tf.matmul(layer_3, weights['out']) + biases['out']


In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=output_layer))
train_step = tf.compat.v1.train.AdamOptimizer(1e-4).minimize(cross_entropy)


In [ ]:
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [ ]:
init = tf.compat.v1.global_variables_initializer()
sess = tf.compat.v1.Session()
sess.run(init)


In [ ]:
# train on mini batches
for i in range(n_iterations):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    sess.run(train_step, feed_dict={X: batch_x, Y: batch_y, keep_prob:dropout})

    # print loss and accuracy (per minibatch)
    if i%100==0:
        minibatch_loss, minibatch_accuracy = sess.run([cross_entropy, accuracy],
                                                      feed_dict={X: batch_x, Y: batch_y, keep_prob:1.0})
        print("Iteration", str(i),
              "\t| Loss =", str(minibatch_loss),
              "\t| Accuracy =", str(minibatch_accuracy))
        

In [ ]:
# train on mini batches
for i in range(n_iterations):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    sess.run(train_step, feed_dict={X: batch_x, Y: batch_y, keep_prob:dropout})

    # print loss and accuracy (per minibatch)
    if i%100==0:
        minibatch_loss, minibatch_accuracy = sess.run([cross_entropy, accuracy], feed_dict={X: batch_x, Y: batch_y, keep_prob:1.0})
        print("Iteration", str(i), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))

In [ ]:
test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob:1.0})
print("\nAccuracy on test set:", test_accuracy)


In [ ]:
import numpy as np
from PIL import Image

In [ ]:
img = np.invert(Image.open("land03.png").convert('L')).ravel()

In [ ]:

prediction = sess.run(tf.argmax(output_layer,1), feed_dict={X: [img]})
print ("Prediction for test image:", np.squeeze(prediction))

In [ ]:
from matplotlib import pyplot as plt # importa pacote para mostrar imagens 
# mostra a primeira imagem no set de treino
plt.imshow(x_train[1].reshape(28,28), cmap='Greys', interpolation='nearest')
#plt.title(str(x_train.labels[1])) # anotação do dígito plt.show()

In [ ]:
lr = 0.01
n_iter = 1000
batch_size = 128
n_inputs = 28 * 28
n_l1 = 512
n_l2 = 512
n_outputs = 10


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)